In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%load_ext autotime

import numpy as np
import pandas as pd
import joblib

import datetime
import os
import numpy as np
import time
import multiprocessing as mp
import re 

In [2]:
import inspect, os
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
os.sys.path.insert(0,parentdir) 
from data_generation.diff_utils import clean_and_filter

time: 51.9 ms


# Load Models

Models are trained on all data unlike paper

In [3]:
from baselines import plurality, average
from serialization import load_pipeline

Using TensorFlow backend.


time: 2.15 s


In [4]:
model_name = 'linear_char_ed_all'
model_type = 'linear_char_ed'
tasks = ['aggression', 'attack']
model_dict = {}
calibrator_dict = {}

for task in tasks:
    path = '../../models/%s/%s' % (task, model_type)
    model_dict[task] = load_pipeline(path, model_name)

time: 15.4 s


In [5]:
def apply_models(df):
    diffs = df['clean_diff']
    for task, model in model_dict.items():
        scores = model.predict_proba(diffs)[:,1]
        df['%s_score' % task] = scores
    return df    

time: 2.34 ms


#### Load samples and apply models

We take various diffs datasets from hive, apply the clean and filter function and the score the clean diffs using the models.

In [6]:
def prep_in_parallel(path, k = 8):
    df = pd.read_csv(path, sep = '\t', encoding = 'utf-8')
    m = df.shape[0] 
    n_groups = int(m / 10000.0)
    df['key'] = np.random.randint(0, high=n_groups, size=m)
    dfs = [e[1] for e in df.groupby('key')]
    dfs = [pred_helper(d) for d in dfs]
    return pd.concat(dfs)

def pred_helper(df):
    if len(df) == 0:
        return None
    return df.assign(rev_timestamp = lambda x: pd.to_datetime(x.rev_timestamp),
            clean_diff = lambda x: x['clean_diff'].astype(str))\
            .pipe(apply_models)

time: 7.21 ms


In [9]:
nss = ['user', 'article']
samples = {'talk_diff_no_admin_2015.tsv': 'model_annotated_comments_2015.tsv',}


for in_name, out_name in samples.items():
    outf = os.path.join('../../data/figshare/', out_name)
    dfs = []
    for ns in nss:
        inf = os.path.join('../../data/samples', ns, 'clean', in_name)
        df = prep_in_parallel(inf, k = 4)
        df['ns'] = ns
        dfs.append(df)
        
    df = pd.concat(dfs)
    df = df.rename(columns={'clean_diff': 'comment',
                        'diff': 'raw_comment',
                        'rev_timestamp': 'timestamp',
                        'rev_id': 'comment_id',
                        'pred_aggression_score_calibrated': 'aggression_score',
                        'pred_attack_score_uncalibrated' : 'attack_score'
                       })
    order = ['comment_id', 'comment', 'raw_comment', 'timestamp', 'page_id', 'page_title', 'user_id', 'user_text', 'ns', 'attack_score', 'aggression_score' ]
    df = df[order]
    df = df.sort_values('timestamp')
    df.to_csv(outf, sep = '\t')   

time: 2h 56min 10s
